In [1]:
import numpy as np
from numpy.random import default_rng
from scipy.special import expit as sigmoid
from scipy.optimize import minimize
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
rng = default_rng(0)

# ---- Synthetic dataset ----
X0 = rng.normal([-1, -1], 0.8, size=(50, 2))
X1 = rng.normal([1, 1], 0.8, size=(50, 2))
X = np.vstack([X0, X1])
y = np.hstack([np.zeros(50), np.ones(50)])

In [3]:
X_design = np.hstack([np.ones((X.shape[0], 1)), X])

In [4]:
perm = rng.permutation(len(y))
X_design, X, y = X_design[perm], X[perm], y[perm]
n_train = 80
X_train, y_train = X_design[:n_train], y[:n_train]
X_test, y_test = X_design[n_train:], y[n_train:]

In [5]:
def nlp(w, X, y, sigma2=5.0):
    z = X @ w
    return -np.sum(y*np.log(sigmoid(z)+1e-9) + (1-y)*np.log(1-sigmoid(z)+1e-9)) + 0.5*np.sum(w**2)/sigma2

def grad_nlp(w, X, y, sigma2=5.0):
    p = sigmoid(X @ w)
    return X.T @ (p - y) + w/sigma2

In [6]:
res = minimize(nlp, np.zeros(X_train.shape[1]), args=(X_train, y_train), jac=grad_nlp)
w_map = res.x

p_test_blr = sigmoid(X_test @ w_map)
y_pred_blr = (p_test_blr >= 0.5).astype(int)
acc_blr = accuracy_score(y_test, y_pred_blr)

In [7]:
print("Bayesian Logistic Regression (Laplace) accuracy:", acc_blr)

Bayesian Logistic Regression (Laplace) accuracy: 1.0
